In [1]:
import pandas as pd
plays = pd.read_csv('NFLPlaybyPlay2015.csv', low_memory = False)
plays #verify data is here

,Unnamed: 0,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,...,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,Season
0,36,2015-09-10,2015091000,1,1,NaN,15:00,15,3600.0,0.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
1,51,2015-09-10,2015091000,1,1,1.0,15:00,15,3600.0,0.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
2,72,2015-09-10,2015091000,1,1,1.0,14:21,15,3561.0,39.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
3,101,2015-09-10,2015091000,1,1,2.0,14:04,15,3544.0,17.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
4,122,2015-09-10,2015091000,1,1,1.0,13:26,14,3506.0,38.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
5,159,2015-09-10,2015091000,1,1,1.0,12:42,13,3462.0,44.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
6,180,2015-09-10,2015091000,1,1,1.0,12:05,13,3425.0,37.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
7,199,2015-09-10,2015091000,1,1,2.0,11:20,12,3380.0,45.0,...,1,PIT,NaN,M.Gilbert,10,0.0,0.0,0.0,0.0,2015
8,236,2015-09-10,2015091000,1,1,2.0,10:53,11,3353.0,27.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
9,261,2015-09-10,2015091000,1,1,3.0,10:28,11,3328.0,25.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015


In [2]:
q4_plays = plays[plays['qtr'] == 4]
q4_plays_twomin = q4_plays[q4_plays['TimeSecs'] <= 120] #split to only be two minute drill
two_min_drill = q4_plays_twomin[q4_plays_twomin['AbsScoreDiff'] <= 14] # 14 arbitrarily defined as the most you can score
gameID = set(two_min_drill['GameID']) #189 games out of 256 apply
data = list()
for x in gameID:
    game_plays = two_min_drill[two_min_drill['GameID'] == x] #split by game
    posTeams = set(game_plays['posteam'])
    for team in posTeams:
        playsbyteam = game_plays[game_plays['posteam'] == team] #split by team
        for sd in set(playsbyteam['ScoreDiff']):
            playbysd = playsbyteam[playsbyteam['ScoreDiff'] == sd] # split by score difference
            passPlays = sum(playbysd.PlayType.str.count('Pass')) + sum(playbysd.PlayType.str.count('Sack')) #assume a sack is a pass
            runPlays = sum(playbysd.PlayType.str.count('Run')) + sum(playbysd.PlayType.str.count('QB Kneel')) #QB Kneel counts as a run
            p_over_r = passPlays - runPlays
            data.append([team, sd, p_over_r])
team_list = list()
sd_list = list()
p_over_r_list = list()
for x in data:
    team_list.append(x[0])
    sd_list.append(x[1])
    p_over_r_list.append(x[2])

In [3]:
from bokeh.io import output_notebook, show
output_notebook()


Loading BokehJS ...

In [4]:
from bokeh.charts import Scatter
#make data frame
df = pd.DataFrame({
        'team':team_list,
        'sd':sd_list,
        'pr':p_over_r_list
    })
colors = ["#97233F", "#C9233F", "#2A0365", "#0F4589", "#0088D4", "#DF6108", "#F03A16", "#322820",
  "#0D254C", "#DF6108", "#006DB0", "#313F36", "#B20032", "#163F83", "#007198", "#B20032",
 "#005E6A", "#3B0160", "#0D254C", "#C9B074", "#192E6C", "#313F36", "#C4C8CB", "#003B48",
 "#F2C800", "#0B264D", "#54BA4C", "#840026", "#0D254C", "#B20032", "#4C96C4", "#7A2D39"] #colors
i = 0
a = set(team_list)
ts = list(a)
ts = sorted(ts)
    
p = Scatter(df, x='pr', y='sd', color="#97233F", title="Score Difference and Plays Run", legend='top_right',
            xlabel="Pass Plays - Run Plays", ylabel="Score Difference")
show(p)
#no team differentiation, just a basic overview


In [5]:
q4_plays = plays[plays['qtr'] == 4]
q4_plays_twomin = q4_plays[q4_plays['TimeSecs'] <= 120] #split to only be two minute drill
two_min_drill = q4_plays_twomin[q4_plays_twomin['AbsScoreDiff'] <= 14] # 14 arbitrarily defined as the most you can score
gameID = set(two_min_drill['GameID']) #189 games out of 256 apply
data = list()
for x in gameID:
    game_plays = two_min_drill[two_min_drill['GameID'] == x] #split by game
    posTeams = set(game_plays['posteam'])
    for team in posTeams:
        playsbyteam = game_plays[game_plays['posteam'] == team] #split by team
        for sd in set(playsbyteam['ScoreDiff']):
            playbysd = playsbyteam[playsbyteam['ScoreDiff'] == sd] # split by score difference
            passPlays = sum(playbysd.PlayType.str.count('Pass')) + sum(playbysd.PlayType.str.count('Sack')) #assume a sack is a pass
            runPlays = sum(playbysd.PlayType.str.count('Run')) + sum(playbysd.PlayType.str.count('QB Kneel')) #QB Kneel counts as a run
            if (passPlays == 0 and runPlays == 0):
                p_over_r = -1 #ignore this data
            else:
                p_over_r = 2 * (float(passPlays) / float(runPlays + passPlays) - 0.5 )#difference from 50% pass 50% run
            data.append([team, sd, p_over_r])
team_list_ratio = list()
sd_list_ratio = list()
p_over_r_list_ratio = list()
for x in data:
    if(x[2] != -1):
        team_list_ratio.append(x[0])
        sd_list_ratio.append(x[1])
        p_over_r_list_ratio.append(x[2])

In [6]:
#make data frame
df_1 = pd.DataFrame({
        'team':team_list_ratio,
        'sd':sd_list_ratio,
        'pr':p_over_r_list_ratio
    })
    
p2 = Scatter(df_1, x='pr', y='sd', color="#97233F", title="Score Difference and Plays Run", legend='top_right',
            xlabel="2(Pass Plays / Total Plays) - 1", ylabel="Score Difference")
show(p2)
#no team differentiation, just a basic overview

In [12]:
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
dic_color = dict(zip(ts, colors))
colorz = [dic_color[t] for t in team_list]

source = ColumnDataSource(
        data=df
    )
hover = HoverTool(
        tooltips=[
            ("Team", "@team"),
            ("(PR, SD)", "(@pr, @sd)")
        ]
    )

team_subtract = figure(tools=[hover], title = "Score Difference and Plays Run by Team")
team_subtract.xaxis.axis_label = 'Pass Plays - Run Plays'
team_subtract.yaxis.axis_label = 'Score Difference'

team_subtract.circle("pr", "sd", color = colorz, fill_alpha=0.2, size=10, source = source)
show(team_subtract)

In [11]:
from bokeh.plotting import figure
from bokeh.models import HoverTool
dic_color = dict(zip(ts, colors))
colorz = [dic_color[t] for t in team_list]

source = ColumnDataSource(
        data=df_1
    )
hover = HoverTool(
        tooltips=[
            ("Team", "@team"),
            ("(PR, SD)", "(@pr, @sd)")
        ]
    )

team_subtract = figure(tools=[hover], title = "Score Difference and Plays Run by Team")
team_subtract.xaxis.axis_label = 'Pass Plays - Run Plays'
team_subtract.yaxis.axis_label = 'Score Difference'

team_subtract.circle("pr", "sd", color = colorz, fill_alpha=0.2, size=10, source = source)
show(team_subtract)

In [9]:
team_dic = None
team_dic = dict.fromkeys(ts,[])
for x in gameID:
    game_plays = two_min_drill[two_min_drill['GameID'] == x] #split by game
    posTeams = set(game_plays['posteam'])
    for team in posTeams:
        playsbyteam = game_plays[game_plays['posteam'] == team] #split by team
        ls = list()
        for sd in set(playsbyteam['ScoreDiff']):
            playbysd = playsbyteam[playsbyteam['ScoreDiff'] == sd] # split by score difference
            passPlays = sum(playbysd.PlayType.str.count('Pass')) + sum(playbysd.PlayType.str.count('Sack')) #assume a sack is a pass
            runPlays = sum(playbysd.PlayType.str.count('Run')) + sum(playbysd.PlayType.str.count('QB Kneel')) #QB Kneel counts as a run
            p_over_r = passPlays - runPlays
            ls.append([team, sd, p_over_r])
        team_dic[team] = ls
            #team_dic[team].append([team,sd, p_over_r])
print(team_dic)
team_list = list()
sd_list = list()
p_over_r_list = list()
for x in data:
    team_list.append(x[0])
    sd_list.append(x[1])
    p_over_r_list.append(x[2])

{'OAK': [['OAK', -2.0, 0]], 'DAL': [['DAL', 0.0, -1], ['DAL', -7.0, 1], ['DAL', -1.0, 1]], 'WAS': [['WAS', -4.0, 1], ['WAS', -3.0, 0], ['WAS', -10.0, 6]], 'CLE': [['CLE', -7.0, 7]], 'KC': [['KC', -7.0, -1], ['KC', -1.0, 0]], 'DET': [['DET', -12.0, 10]], 'SF': [['SF', -14.0, 2]], 'TEN': [['TEN', -7.0, 4], ['TEN', -1.0, -2]], 'JAC': [['JAC', 0.0, 4]], 'MIN': [['MIN', -3.0, 4]], 'PIT': [['PIT', 6.0, -2]], 'SEA': [['SEA', 1.0, -1], ['SEA', -2.0, -1]], 'STL': [['STL', -6.0, 5]], 'NE': [['NE', 8.0, -2]], 'MIA': [['MIA', -13.0, 3]], 'GB': [['GB', 14.0, -3]], 'BAL': [['BAL', -5.0, 3]], 'ARI': [['ARI', 14.0, -2]], 'HOU': [['HOU', 10.0, -1], ['HOU', 7.0, -1]], 'TB': [['TB', -13.0, 1]], 'NYG': [['NYG', 14.0, -1]], 'CIN': [['CIN', 5.0, -4]], 'PHI': [['PHI', 3.0, 4]], 'CAR': [['CAR', 13.0, -2]], 'SD': [['SD', 0.0, -2], ['SD', 3.0, 0]], 'BUF': [['BUF', -14.0, 8]], 'IND': [['IND', 0.0, 2]], 'DEN': [['DEN', 0.0, 1], ['DEN', 3.0, -1]], 'NO': [['NO', 0.0, -1]], 'CHI': [['CHI', 2.0, 0], ['CHI', -1.0, 8]]